In [ ]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_product_review(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    
    while True:
        
        try:
            wd.get(prd_link)
            time.sleep(5)
            wd.maximize_window()

            socket.setdefaulttimeout(30)
            
            wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
            time.sleep(5)

            html = wd.page_source
            bs = BeautifulSoup(html, 'html.parser')
            time.sleep(3)

            #리뷰
            element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/section')
            wd.execute_script("arguments[0].scrollIntoView(true);", element)
            time.sleep(5)

            review = []
            rating = []
            review_date = []
            user_review_cnt = []
            user_skin_type = []
            user_age = []
            user_sex = []
            user_name = []

            review_len = len(wd.find_elements_by_class_name("cutter__pre"))
            info_details = bs.select(".info__details")
            time.sleep(5)
            
            if review_len == 0:
                review_info = {
                    "리뷰": review,
                    "별점": rating,
                    "리뷰 일자": review_date,
                    "사용자 리뷰 개수": user_review_cnt,
                    "사용자 피부 타입": user_skin_type,
                    "사용자 나이": user_age,
                    "사용자 성별": user_sex,
                    "사용자 이름": user_name,
                }

                return review_info      
            
            star = bs.select(".stars")
            for i in range(review_len):
                if '프로필 미등록' in str(info_details[i]):
                    continue
                review.append(wd.find_elements_by_class_name("cutter__pre")[i].text.replace('\n', ' '))
                review_date.append(bs.select(".review__side-info__created-at")[i].text.strip())
                user_age.append(str(info_details[i]).split('\n')[1].strip())
                user_skin_type.append(str(info_details[i]).split('\n')[3].strip())
                user_sex.append(str(info_details[i]).split('\n')[5].strip())
                user_review_cnt.append(str(info_details[i]).split('\n')[7].strip())
                user_name.append(re.findall('<p class="info__details__nickname" data(.*?)>(.*?)</p>', str(info_details[i]).split('\n')[0])[0][-1])
                rating.append(str(star[i+2]).split('\n')[-2].strip())
                
            time.sleep(3)
            wd.close()

            review_info = {
                "리뷰": review,
                "별점": rating,
                "리뷰 일자": review_date,
                "사용자 리뷰 개수": user_review_cnt,
                "사용자 피부 타입": user_skin_type,
                "사용자 나이": user_age,
                "사용자 성별": user_sex,
                "사용자 이름": user_name,
            }

            return review_info
        
        except urllib3.exceptions.ReadTimeoutError:
            time.sleep(5)
            
        except Exception as e:
            exception.append(prd_link)
            print(e)
            return None

In [ ]:
for i in tqdm(list(fnl_df_glowpick_prd_info.loc[pd.isnull(fnl_df_glowpick_prd_info['리뷰'])].index)):
    review_info = scrap_product_reviews(fnl_df_glowpick_prd_info.loc[i, '제품 link'])
    if review_info != None:
        fnl_df_glowpick_prd_info.loc[i, '리뷰'] = str(review_info['리뷰'])
        fnl_df_glowpick_prd_info.loc[i, '별점'] = str(review_info['별점'])
        fnl_df_glowpick_prd_info.loc[i, '리뷰 일자'] = str(review_info['리뷰 일자'])
        fnl_df_glowpick_prd_info.loc[i, '사용자 리뷰 개수'] = str(review_info['사용자 리뷰 개수'])
        fnl_df_glowpick_prd_info.loc[i, '사용자 피부 타입'] = str(review_info['사용자 피부 타입'])
        fnl_df_glowpick_prd_info.loc[i, '사용자 나이'] = str(review_info['사용자 나이'])
        fnl_df_glowpick_prd_info.loc[i, '사용자 성별'] = str(review_info['사용자 성별'])
        fnl_df_glowpick_prd_info.loc[i, '사용자 이름'] = str(review_info['사용자 이름'])